In [4]:
text_chunks = []
doc_ids = []
nodes = []

In [1]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="https://de17d8d3-22aa-4464-aec7-ab8792b5d72f.us-east4-0.gcp.cloud.qdrant.io:6333/",
    api_key="2xsy0iXGIzI8Fj_Fa7eiTRsb9fyl612Zl1Qm_i-9PpA2YucLSQLytw"
)

In [2]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter

text_parser = SentenceSplitter(chunk_size=512, chunk_overlap=100)
embed_model = OllamaEmbedding(model_name='mxbai-embed-large', base_url='http://localhost:11434')

Settings.embed_model = embed_model
Settings.transformations = [text_parser]

In [3]:
from llama_index.readers.file import PDFReader

docs = PDFReader().load_data(r"C:\Users\nokia\Desktop\Keerthi-X\x physics em.pdf")

In [5]:
print("enumerating docs")
for doc_idx, doc in enumerate(docs):
    curr_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(curr_text_chunks)
    doc_ids.extend([doc_idx] * len(curr_text_chunks))

enumerating docs


In [8]:
from llama_index.core.schema import TextNode

print("enumerating text_chunks")
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(text=text_chunk)
    src_doc = docs[doc_ids[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

enumerating text_chunks


In [12]:
print(f"No. of nodes : {len(nodes)}")

No. of nodes : 408


In [15]:
from llama_index.core.schema import MetadataMode

print("enumerating nodes")
count = 0
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode=MetadataMode.ALL)
    )
    node.embedding = node_embedding
    print(f"Indexing {count}")
    count += 1

enumerating nodes
Indexing 0
Indexing 1
Indexing 2
Indexing 3
Indexing 4
Indexing 5
Indexing 6
Indexing 7
Indexing 8
Indexing 9
Indexing 10


In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

print("initializing the storage context")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print("indexing the nodes in VectorStoreIndex")
index = VectorStoreIndex(
    nodes=nodes,
    storage_context=storage_context,
    transformations=Settings.transformations,
)